<a href="https://colab.research.google.com/github/Jucivania-cp/Projeto_Integrado_II/blob/main/Projeto_Integrado_II.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sidrapy

#IMPORTAÇÃO DOS DADOS

##Dados de produção do milho

In [40]:
import pandas as pd
import sidrapy
import requests
import os

# 1. Configurações Iniciais e Códigos dos Municípios (IBGE)
# Lista: Juazeiro, Crato, Barbalha, Jardim, Missão Velha,
# Caririaçu, Farias Brito, Nova Olinda e Santana do Cariri.
municipios_codes = "2307304,2304202,2301901,2307106,2308401,2303204,2304301,2309201,2312106"

def get_base_producao_agricola():
    """Coleta dados da Tabela 839 do SIDRA (Milho) com códigos corrigidos"""
    print("Coletando Base 1: Produção Agrícola (Tabela 839)...")

    # v109 (Plantada), v216 (Colhida), v214 (Produzida), v112 (Rendimento)
    try:
        df_producao = sidrapy.get_table(
                      table_code="839",
                      territorial_level="6",
                      ibge_territorial_code=municipios_codes, # Corrected parameter name
                      period="2014-2023",
                      variable="109,216,214,112"
        )

        # Limpeza de Metadados (Remover a primeira linha que contém descrições)
        df_producao.columns = df_producao.iloc[0]
        df_producao = df_producao.iloc[1:]

        # Treatment of missing values and conversion to numeric
        # SIDRA uses '...' for unavailable data
        # Identify the value column, assuming it's the last column after metadata cleaning
        value_col_name = df_producao.columns[-1]
        df_producao[value_col_name] = pd.to_numeric(df_producao[value_col_name].replace(['...', '-'], '0'), errors='coerce')

        df_producao.to_csv("base1_producao_milho_corrigida.csv", index=False)
        print("Base 1 salva com sucesso com códigos 109, 216, 214 e 112.")
        return df_producao

    except Exception as e:
        print(f"Erro na coleta da Tabela 839: {e}")



##Dados do PIB

In [ ]:
def get_base_pib():
    """Coleta dados de PIB Municipal (Série 2010-2023)"""
    print("Coletando Base 3: Indicadores Socioeconômicos (PIB)...")
    # Tabela 5938 - PIB a preços correntes e PIB per capita
    try:
        df_pib = sidrapy.get_table(
                table_code="5938",
                territorial_level="6",
                ibge_territorial_code=municipios_codes,
                period="2014-2021", # Nota: PIB tem defasagem de 2 anos no IBGE
                variable="37" # Removed '39' as it was causing an error
        )
        df_pib.to_csv("base3_pib_socioeconomico.csv", index=False)
        print("Base 3 salva com sucesso.")
        return df_pib
    except Exception as e:
        print(f"Erro na coleta da Tabela 5938 (PIB): {e}")


###Visualisar tabela

In [41]:
df_pib = get_base_pib()
df_pib.head()

Coletando Base 3: Indicadores Socioeconômicos (PIB)...
Base 3 salva com sucesso.


,NC,NN,MC,MN,V,D1C,D1N,D2C,D2N,D3C,D3N
0,Nível Territorial (Código),Nível Territorial,Unidade de Medida (Código),Unidade de Medida,Valor,Município (Código),Município,Ano (Código),Ano,Variável (Código),Variável
1,6,Município,40,Mil Reais,3819082,2307304,Juazeiro do Norte (CE),2014,2014,37,Produto Interno Bruto a preços correntes
2,6,Município,40,Mil Reais,3921616,2307304,Juazeiro do Norte (CE),2015,2015,37,Produto Interno Bruto a preços correntes
3,6,Município,40,Mil Reais,4183987,2307304,Juazeiro do Norte (CE),2016,2016,37,Produto Interno Bruto a preços correntes
4,6,Município,40,Mil Reais,4428414,2307304,Juazeiro do Norte (CE),2017,2017,37,Produto Interno Bruto a preços correntes


In [34]:
import pandas as pd
import requests
import zipfile
import os

def baixar_e_extrair_funceme(url_zip, pasta_destino="dados_funceme"):
    """
    Baixa o arquivo ZIP da FUNCEME e extrai seu conteúdo automaticamente.
    """
    arquivo_zip = "postos.zip"

    print(f"Iniciando o download de: {url_zip}...")

    try:
        # 1. Faz o download do arquivo
        response = requests.get(url_zip, stream=True)
        response.raise_for_status() # Verifica se o link está ativo

        with open(arquivo_zip, "wb") as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)

        print("Download concluído! Extraindo arquivos...")

        # 2. Extrai o conteúdo do ZIP
        if not os.path.exists(pasta_destino):
            os.makedirs(pasta_destino)

        with zipfile.ZipFile(arquivo_zip, 'r') as zip_ref:
            zip_ref.extractall(pasta_destino)

        # 3. Limpeza: remove o arquivo .zip após extrair
        os.remove(arquivo_zip)

        print(f"Arquivos extraídos com sucesso na pasta: '{pasta_destino}'")
        return pasta_destino

    except Exception as e:
        print(f"Erro ao automatizar o download: {e}")
        return None

# Execução automática
url = "https://cdn.funceme.br/calendario/postos/postos.zip"
pasta_dos_csvs = baixar_e_extrair_funceme(url)

def processar_arquivos_por_id(caminho_pasta):
    # Dicionário de mapeamento (Exemplo: preencha com os IDs que você encontrar no site)
    # Formato: 'ID_DO_ARQUIVO': 'NOME_DO_MUNICIPIO'
    mapeamento_cariri = {
        '78': 'JUAZEIRO DO NORTE',
        '43': 'CRATO',
        '20': 'BARBALHA',
        '76': 'JARDIM',
        '91': 'MISSAO VELHA',
        '33': 'CARIRIACU',
        '45': 'FARIAS BRITO',
        '99': 'NOVA OLINDA',
        '131': 'SANTANA DO CARIRI'
    }

    lista_dfs = []

    print("Iniciando leitura dos arquivos por ID...")

    for id_posto, nome_cidade in mapeamento_cariri.items():
        arquivo_nome = f"{id_posto}.txt"
        caminho_completo = os.path.join(caminho_pasta, arquivo_nome)

        if os.path.exists(caminho_completo):
            # Ler o TXT
            df_temp = pd.read_csv(caminho_completo, sep=';', encoding='latin-1')
            # Diagnóstico: Imprimir os nomes das colunas
            print(f"Colunas em {arquivo_nome} ({nome_cidade}): {df_temp.columns.tolist()}")

            # Adicionar coluna com o nome do município para identificação
            df_temp['Municipio'] = nome_cidade
            lista_dfs.append(df_temp)
            print(f"Arquivo {arquivo_nome} ({nome_cidade}) carregado.")
        else:
            print(f"Aviso: Arquivo {arquivo_nome} não encontrado para {nome_cidade}.")

    if lista_dfs:
        df_raw = pd.concat(lista_dfs, ignore_index=True)

        # Melt the 'Dia' columns into a single column
        id_vars = ['Municipios', 'Postos', 'Latitude', 'Longitude', 'Anos', 'Meses', 'Total', 'Municipio']
        value_vars = [col for col in df_raw.columns if col.startswith('Dia') and col[3:].isdigit()]

        df_melted = df_raw.melt(
            id_vars=id_vars,
            value_vars=value_vars,
            var_name='Dia_str',
            value_name='valor_chuva_mm'
        )

        # Extract day number from 'Dia_str' (e.g., 'Dia1' -> '1')
        df_melted['Dia'] = df_melted['Dia_str'].str.replace('Dia', '', regex=False).astype(int)

        # Create 'Data' column
        # Pad month and day with leading zeros for proper date parsing
        df_melted['Data'] = pd.to_datetime(
            df_melted['Anos'].astype(str) + '-' +
            df_melted['Meses'].astype(str).str.zfill(2) + '-' +
            df_melted['Dia'].astype(str).str.zfill(2),
            errors='coerce' # Coerce errors to NaT for invalid dates (e.g., Feb 30)
        )

        # Filter out rows with invalid dates (NaT)
        df_melted.dropna(subset=['Data'], inplace=True)

        # Filter period from 2014 to 2024
        df_final_local = df_melted[(df_melted['Data'].dt.year >= 2014) & (df_melted['Data'].dt.year <= 2024)]

        # Select and reorder relevant columns
        df_final_local = df_final_local[['Data', 'Municipio', 'valor_chuva_mm']]

        return df_final_local
    else:
        return None

#Processamento dos arquivos
if pasta_dos_csvs:
    # Caso os CSVs estejam em uma subpasta 'postos' dentro da extração:
    caminho_real = os.path.join(pasta_dos_csvs, "postos")
    if not os.path.exists(caminho_real):
        caminho_real = pasta_dos_csvs # Se não houver subpasta, usa a principal

    #df_final = processar_arquivos_por_id(caminho_real)

    #if df_final is not None:
        #print("\nBase Pluviométrica do Cariri gerada com sucesso!")
        #display(df_final.head())
    #else:
        #print("\nErro: Nenhum dado foi processado. Verifique os IDs e o caminho dos arquivos.")

Iniciando o download de: https://cdn.funceme.br/calendario/postos/postos.zip...
Download concluído! Extraindo arquivos...
Arquivos extraídos com sucesso na pasta: 'dados_funceme'


In [35]:
# Execução do pipeline
if __name__ == "__main__":
    try:
        base1 = get_base_producao_agricola()
        base3 = get_base_pib()
        base2 = processar_arquivos_por_id(caminho_real)
        print("\nPipeline concluído com sucesso! Arquivos CSV salvos no diretório.")
    except Exception as e:
        print(f"Erro na coleta: {e}")

Coletando Base 1: Produção Agrícola (Tabela 839)...
Base 1 salva com sucesso com códigos 109, 216, 214 e 112.
Coletando Base 3: Indicadores Socioeconômicos (PIB)...
Base 3 salva com sucesso.
Iniciando leitura dos arquivos por ID...
Colunas em 78.txt (JUAZEIRO DO NORTE): ['Municipios', 'Postos', 'Latitude', 'Longitude', 'Anos', 'Meses', 'Total', 'Dia1', 'Dia2', 'Dia3', 'Dia4', 'Dia5', 'Dia6', 'Dia7', 'Dia8', 'Dia9', 'Dia10', 'Dia11', 'Dia12', 'Dia13', 'Dia14', 'Dia15', 'Dia16', 'Dia17', 'Dia18', 'Dia19', 'Dia20', 'Dia21', 'Dia22', 'Dia23', 'Dia24', 'Dia25', 'Dia26', 'Dia27', 'Dia28', 'Dia29', 'Dia30', 'Dia31']
Arquivo 78.txt (JUAZEIRO DO NORTE) carregado.
Colunas em 43.txt (CRATO): ['Municipios', 'Postos', 'Latitude', 'Longitude', 'Anos', 'Meses', 'Total', 'Dia1', 'Dia2', 'Dia3', 'Dia4', 'Dia5', 'Dia6', 'Dia7', 'Dia8', 'Dia9', 'Dia10', 'Dia11', 'Dia12', 'Dia13', 'Dia14', 'Dia15', 'Dia16', 'Dia17', 'Dia18', 'Dia19', 'Dia20', 'Dia21', 'Dia22', 'Dia23', 'Dia24', 'Dia25', 'Dia26', 'Dia27', 

In [37]:
base2.head()

#os.listdir(pasta_dos_csvs)

,Data,Municipio,valor_chuva_mm
480,2014-01-01,JUAZEIRO DO NORTE,0.0
481,2014-02-01,JUAZEIRO DO NORTE,0.0
482,2014-03-01,JUAZEIRO DO NORTE,0.0
483,2014-04-01,JUAZEIRO DO NORTE,0.0
484,2014-05-01,JUAZEIRO DO NORTE,0.0


In [38]:
base1.head()

,Nível Territorial (Código),Nível Territorial,Unidade de Medida (Código),Unidade de Medida,Valor,Município (Código),Município,Ano (Código),Ano,Variável (Código),Variável,Produto das lavouras temporárias (Código),Produto das lavouras temporárias
1,6,Município,1006,Hectares,633,2307304,Juazeiro do Norte (CE),2014,2014,109,Área plantada,31693,NaN
2,6,Município,1006,Hectares,633,2307304,Juazeiro do Norte (CE),2014,2014,216,Área colhida,31693,NaN
3,6,Município,1017,Toneladas,2507,2307304,Juazeiro do Norte (CE),2014,2014,214,Quantidade produzida,31693,NaN
4,6,Município,33,Quilogramas por Hectare,3961,2307304,Juazeiro do Norte (CE),2014,2014,112,Rendimento médio da produção,31693,NaN
5,6,Município,1006,Hectares,664,2307304,Juazeiro do Norte (CE),2015,2015,109,Área plantada,31693,NaN


In [39]:
base3.head()

,NC,NN,MC,MN,V,D1C,D1N,D2C,D2N,D3C,D3N
0,Nível Territorial (Código),Nível Territorial,Unidade de Medida (Código),Unidade de Medida,Valor,Município (Código),Município,Ano (Código),Ano,Variável (Código),Variável
1,6,Município,40,Mil Reais,3819082,2307304,Juazeiro do Norte (CE),2014,2014,37,Produto Interno Bruto a preços correntes
2,6,Município,40,Mil Reais,3921616,2307304,Juazeiro do Norte (CE),2015,2015,37,Produto Interno Bruto a preços correntes
3,6,Município,40,Mil Reais,4183987,2307304,Juazeiro do Norte (CE),2016,2016,37,Produto Interno Bruto a preços correntes
4,6,Município,40,Mil Reais,4428414,2307304,Juazeiro do Norte (CE),2017,2017,37,Produto Interno Bruto a preços correntes


In [ ]:
#import pandas as pd

# Carregar as bases
df_producao = pd.read_csv("base1_producao_milho_corrigida.csv")
df_pib = pd.read_csv("base3_pib_socioeconomico.csv")

# Visualizar as 5 primeiras linhas da base de produção
print("--- Base de Produção Agrícola (Milho) ---")
display(df_producao.head())

# Visualizar as 5 primeiras linhas da base de PIB
print("\n--- Base de Indicadores Socioeconômicos (PIB) ---")
display(df_pib.head())


--- Base de Produção Agrícola (Milho) ---


,Nível Territorial (Código),Nível Territorial,Unidade de Medida (Código),Unidade de Medida,Valor,Município (Código),Município,Ano (Código),Ano,Variável (Código),Variável,Produto das lavouras temporárias (Código),Produto das lavouras temporárias
0,6,Município,1006,Hectares,633,2307304,Juazeiro do Norte (CE),2014,2014,109,Área plantada,31693,NaN
1,6,Município,1006,Hectares,633,2307304,Juazeiro do Norte (CE),2014,2014,216,Área colhida,31693,NaN
2,6,Município,1017,Toneladas,2507,2307304,Juazeiro do Norte (CE),2014,2014,214,Quantidade produzida,31693,NaN
3,6,Município,33,Quilogramas por Hectare,3961,2307304,Juazeiro do Norte (CE),2014,2014,112,Rendimento médio da produção,31693,NaN
4,6,Município,1006,Hectares,664,2307304,Juazeiro do Norte (CE),2015,2015,109,Área plantada,31693,NaN



--- Base de Indicadores Socioeconômicos (PIB) ---


,NC,NN,MC,MN,V,D1C,D1N,D2C,D2N,D3C,D3N
0,Nível Territorial (Código),Nível Territorial,Unidade de Medida (Código),Unidade de Medida,Valor,Município (Código),Município,Ano (Código),Ano,Variável (Código),Variável
1,6,Município,40,Mil Reais,3819082,2307304,Juazeiro do Norte (CE),2014,2014,37,Produto Interno Bruto a preços correntes
2,6,Município,40,Mil Reais,3921616,2307304,Juazeiro do Norte (CE),2015,2015,37,Produto Interno Bruto a preços correntes
3,6,Município,40,Mil Reais,4183987,2307304,Juazeiro do Norte (CE),2016,2016,37,Produto Interno Bruto a preços correntes
4,6,Município,40,Mil Reais,4428414,2307304,Juazeiro do Norte (CE),2017,2017,37,Produto Interno Bruto a preços correntes
